## Estimate WM fractions (0-1) in the top 1000m of the Arctic Ocean using T, S, DO observations

In [1]:
# Load Arctic T, S, DO data

import pandas as pd
import numpy as np
import xarray as xr

data_path = '/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/Arctic_T_S_DO_data.nc'
arctic = xr.open_dataset(data_path).to_dataframe()

/Users/ko389/miniconda3/envs/hydro_env_local/lib/python3.12/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


In [2]:
# Data preparation

def set_up_arctic_data(df):

    # rename & reorder columns
    df=df.rename(columns={'dissolved_oxygen':'oxygen','conservative_temperature':'conservative_temp'})
    column_order = ['conservative_temp', 'absolute_salinity', 'oxygen','latitude', 'longitude', 'depth', 'source','nprof', 'datetime']
    df = df [column_order]

    # remove labrador sea
    labrador_Sea = df[(df['latitude']<80)&(df['longitude']<-40)&(df['longitude']>-120)].index
    df = df.drop(labrador_Sea)

    # remove data with depth > 1000m
    df = df[df['depth']<1000]
    df = df.dropna(subset=['conservative_temp','absolute_salinity','oxygen'])
    
    return df

arctic = set_up_arctic_data(arctic)

In [3]:
# Make a copy of data and add mass column
df = arctic.copy()  
df["mass"] = 1.0

# Convert datetime column to string

In [ ]:
# End members for source water types

# ASW: Arctic Surface Water
# sPW: Alaskan Coastal Current Water
# MMsPW: Modified summer Pacific Water
# wPW: winter Pacific Water
# NCW: Norwegian Current Water
# Modified NCW: Modified Atlantic Water
# BW: Brine-enriched Water

#Seasonal modification of end members:
#ASW in summer is T=-1.35, S=25.5, DO=367.25
#NCW in summer is T=12, S=35.3, DO=275.56
#MsPW is removed in winter

values = {
    'ASW_T': -1.45, 
    'ASW_S': 27,
    'ASW_O': 390.53,
    #'MsPW_T': 0.5, #Remove to test
    #'MsPW_S': 31,
    #'MsPW_O': 354.21,
    'sPW_T': 6,
    'sPW_S': 31.2,
    'sPW_O': 314,
    'wPW_T': -1.5,
    'wPW_S': 33.2,
    'wPW_O': 280.84,
    'NCW_T': 8, 
    'NCW_S': 35.4,
    'NCW_O': 287.82,
    'AW_T': 0.0, #####0.3 using post-2010 data######
    'AW_S': 35.05,
    'AW_O': 298,
    'BW_T': -1.8, #####-1.75 using post-2010 data######
    'BW_S': 34.45,
    'BW_O': 370
}


In [ ]:
ASW = ["ASW", values['ASW_T'], values['ASW_S'], values['ASW_O']]    
#MsPW = ["MsPW", values['MsPW_T'], values['MsPW_S'], values['MsPW_O']]
sPW = ["sPW", values['sPW_T'], values['sPW_S'], values['sPW_O']]
wPW = ["wPW", values['wPW_T'], values['wPW_S'], values['wPW_O']] 
NCW = ["NCW ", values['NCW_T'], values['NCW_S'], values['NCW_O']] 
AW = ["AW", values['AW_T'], values['AW_S'], values['AW_O']] 
BW = ["BW", values['BW_T'], values['BW_S'], values['BW_O']] 

def prepare_endmember_df(endmembers_arr):
    df = pd.DataFrame(data=endmembers_arr,
                      columns=["endmember_name", "conservative_temp", "absolute_salinity","oxygen"
                               ])
    df["mass"] = 1
    return df

endmemberdf = prepare_endmember_df(
    [ASW,sPW,wPW,NCW,AW,BW]) #Add back MsPW if needed

endmemberdf 

#####REMOVE - no oxygen end members dataframe######
#endmemberdf_no_oxygen = endmemberdf.drop(columns=['oxygen'])
#endmemberdf_no_oxygen 

,endmember_name,conservative_temp,absolute_salinity,oxygen,mass
0,ASW,-1.45,27.00,390.53,1
1,ACCW,6.00,31.20,314.00,1
2,wPW,-1.50,33.20,280.84,1
3,NCW,8.00,35.40,287.82,1
4,MAW,0.00,35.05,298.00,1
5,BW,-1.80,34.45,370.00,1


In [ ]:
# Run code in chunks

import pyompa
from pyompa import (
    plot_ompasoln_endmember_fractions,
    plot_ompasoln_residuals,
    plot_ompasoln_endmember_usagepenalties
)

# Assuming 'df' is your DataFrame with 1522168 rows

chunk_size = 10000
num_chunks = len(df) // chunk_size + 1

# Define column titles
column_titles = ["ASW", "sPW","wPW","NCW","AW","BW"] # Add back in "MsPW" if needed

# Initialize empty DataFrames to store results
all_wm_fractions = pd.DataFrame(columns=column_titles)
all_residuals = pd.DataFrame(columns=['conservative_temp', 'absolute_salinity', 'oxygen']) 

convertedparamgroups = [
    pyompa.ConvertedParamGroup(
        groupname="phosphate_remin",
        conversion_ratios=[{"oxygen": -170, "phosphate": 1.0}],
        always_positive=False
    )
]

paramweightings = {
    "conservative_temp": 24,
    "absolute_salinity": 24,
    "oxygen": 7, 
    "mass": 24
}

hardmasscons_settings = {
    "param_names": ["conservative_temp", "absolute_salinity","oxygen","mass"], 
    "param_weightings": paramweightings,
    "convertedparam_groups": convertedparamgroups,
    "sumtooneconstraint": True,
    "standardize_by_watertypes": True
}

# Loop over chunks
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    
    # Extract chunk of data
    chunk_df = df.iloc[start_idx:end_idx]
    
     # Run the analysis with the hard mass conservation constraint for the current chunk
    ompasol = pyompa.OMPAProblem(
        obs_df=chunk_df.dropna(),
        **hardmasscons_settings
    ).solve(
        endmemberdf,
        endmember_name_column="endmember_name"
    )
    
    # Concatenate results to the overall DataFrames
    all_wm_fractions = pd.concat([all_wm_fractions, pd.DataFrame(ompasol.endmember_fractions, columns=column_titles)])
    all_residuals = pd.concat([all_residuals, pd.DataFrame(ompasol.param_residuals, columns=ompasol.param_names)])

# Reset index for the final DataFrame
all_wm_fractions.reset_index(drop=True, inplace=True)
all_residuals.reset_index(drop=True, inplace=True)

# Display the final DataFrames
print("All WM Fractions:")
print(all_wm_fractions)

print("\nAll Residuals:")
print(all_residuals)


Endmember-idx mapping is
 OrderedDict({'ASW': [0], 'ACCW': [1], 'wPW': [2], 'NCW ': [3], 'MAW': [4], 'BW': [5]})
I'm assuming that the index encoding mass is: [3]
Std used for normalization: [ 4.32046487  3.18946181 45.80109799  1.        ]
Mean used for normalization: [  1.54166667  32.71666667 323.53166667   0.        ]
params to use: ['conservative_temp', 'absolute_salinity', 'oxygen', 'mass']
param weighting: [24 24  7 24]
effective weighting: [ 5.5549578   7.52478048  0.15283476 24.        ]
Matrix A:
Trying convertedvariable sign constraint: [1]
On example 0 to 10000 out of 10000
status: optimal
optimal value 189157.41209492637
Original weighted sum squares: 189157.41209492637
Post fix weighted sum squared: 189161.5728192014
Trying convertedvariable sign constraint: [-1]
On example 0 to 10000 out of 10000
status: optimal
optimal value 469742.5046718186
Original weighted sum squares: 469742.5046718186
Post fix weighted sum squared: 469746.2265845589
On example 0 to 10000 out of 10

/var/folders/zn/185xql1x357249zc9q1yjqvm0000gq/T/ipykernel_5575/1703813259.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_wm_fractions = pd.concat([all_wm_fractions, pd.DataFrame(ompasol.endmember_fractions, columns=column_titles)])
/var/folders/zn/185xql1x357249zc9q1yjqvm0000gq/T/ipykernel_5575/1703813259.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_residuals = pd.concat([all_residuals, pd.DataFrame(ompasol.param_residuals, columns=ompasol.param_names)])


status: optimal
optimal value 270697.9893819674
Original weighted sum squares: 270697.9893819674
Post fix weighted sum squared: 270697.99582271586
Trying convertedvariable sign constraint: [-1]
On example 0 to 10000 out of 10000
status: optimal
optimal value 246259.2020505166
Original weighted sum squares: 246259.2020505166
Post fix weighted sum squared: 246264.41356938402
On example 0 to 10000 out of 10000
status: optimal
optimal value 225832.33187869954
Original weighted sum squares: 225832.33187869954
Post fix weighted sum squared: 225881.9406337508
objective: 225881.9406337508
Endmember-idx mapping is
 OrderedDict({'ASW': [0], 'ACCW': [1], 'wPW': [2], 'NCW ': [3], 'MAW': [4], 'BW': [5]})
I'm assuming that the index encoding mass is: [3]
Std used for normalization: [ 4.32046487  3.18946181 45.80109799  1.        ]
Mean used for normalization: [  1.54166667  32.71666667 323.53166667   0.        ]
params to use: ['conservative_temp', 'absolute_salinity', 'oxygen', 'mass']
param weight

In [16]:
all_residuals

,conservative_temp,absolute_salinity,oxygen,mass
0,-3.552714e-15,-7.105427e-15,0.000000e+00,-1.110223e-16
1,-2.664535e-15,-7.105427e-15,5.684342e-14,-1.110223e-16
2,-3.552714e-15,0.000000e+00,5.684342e-14,0.000000e+00
3,-2.664535e-15,0.000000e+00,0.000000e+00,0.000000e+00
4,-3.552714e-15,7.105427e-15,5.684342e-14,2.220446e-16
...,...,...,...,...
1179487,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1179488,0.000000e+00,-7.105427e-15,-5.684342e-14,-1.110223e-16
1179489,6.000000e-02,-3.000000e-02,-1.358259e-03,0.000000e+00
1179490,5.900000e-02,-3.000000e-02,-1.342130e-03,0.000000e+00


In [ ]:
# Combine the results with the original data

arctic.reset_index(drop=True, inplace=True)
all_wm_fractions = pd.concat([arctic, all_wm_fractions[['ASW', 'NCW', 'AW', 'sPW','wPW', 'BW']]],axis=1) #Add back 'MsPW' if needed

#rename residuals columns
#all_residuals.columns = ['conservative_temp_residuals', 'absolute_salinity_residuals', 'oxygen_residuals', 'mass_residuals']
#arctic.reset_index(drop=True, inplace=True)
#all_residuals= pd.concat([arctic, all_residuals],axis=1) 

In [9]:
# Ensure 'datetime' is properly formatted as a coordinate
df = all_wm_fractions.copy()
df['datetime'] = pd.to_datetime(df['datetime'])

# Convert to xarray Dataset (without setting too many indices)
ds = xr.Dataset.from_dataframe(df)

# Save to NetCDF
output_path = "/Users/ko389/Documents/GitHub/Arctic_water_masses/Data/OMP_output_no_MsPW.nc"#_without_DO.nc"
ds.to_netcdf(output_path, engine="netcdf4")